In [52]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models
import glob
from sklearn.model_selection import train_test_split
import sklearn

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

# class_names = {"A":0,"B":1,"C":2,"D":3,"E":4,"F":5,"G":6,"H":7,"I":8,"J":9,"K":10,"L":11,"M":12,"N":13,"O":14,"P":15,"Q":16,"R":17,"S":18,"T":19,"U":20,"V":21,"W":22,"X":23,"Y":24,"Z":25}
model_fp = "C:\\Users\\david\\Documents\\David\\MSU\\CSC450\\landmark_mlp_model_v1.h5"
mod_csv = "C:\\Users\\david\\Documents\\David\\MSU\\CSC450\\parsed_mod_dataset.csv"
train_fp = "C:\\Users\\david\\Documents\\David\\MSU\\CSC450\\mod_train.csv"
test_fp = "C:\\Users\\david\\Documents\\David\\MSU\\CSC450\\mod_test.csv"

TensorFlow version: 2.4.0-dev20200902
Eager execution: True


In [56]:
def createDataset(dataset_fp):
    df = pd.read_csv(dataset_fp)
    column_names = df.columns.tolist()[:1]
    label_name = df.columns.tolist()[0]
    batch_size = 63
    train_dataset = tf.data.experimental.make_csv_dataset(
        dataset_fp,
        batch_size,
        column_names=column_names,
        label_name=label_name,
        num_epochs=1)
    return dataset

In [9]:
# def createDatasetFromGlob(files):
#     dataset = tf.data.Dataset.list_files(files)  # doctest: +SKIP
#     return dataset

In [33]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

In [34]:
def create():
    # METRICS = [
    #   tf.keras.metrics.TruePositives(name='tp'),
    #   tf.keras.metrics.FalsePositives(name='fp'),
    #   tf.keras.metrics.TrueNegatives(name='tn'),
    #   tf.keras.metrics.FalseNegatives(name='fn')]
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(63,)),
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(26, activation=tf.nn.softmax)])
    model.compile(optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=tf.keras.metrics.SparseCategoricalAccuracy())
    return model

In [48]:
mod_ds = pd.read_csv(mod_csv)
mod_ds = sklearn.S
train_ds, test_ds = train_test_split(mod_ds)
train_ds.to_csv(train_fp, index=None)
test_ds.to_csv(test_fp, index=None)

In [57]:
train_ds = createDataset(train_fp)
train_ds = train_ds.map(pack_features_vector)
test_ds = createDataset(test_fp)
test_ds = test_ds.map(pack_features_vector)
model = create()
model.fit(train_ds, epochs=150)

ValueError: Problem inferring types: CSV row has different number of fields than expected.

In [67]:
loss, acc = model.evaluate(test_ds, verbose=2)
# loss, tp, fp, tn, fn = model.evaluate(test_ds, verbose=0)
print("Model, accuracy:", acc)
print("Model, loss:", loss)
# print("Model, tp:", tp)
# print("Model, fp:", fp)
# print("Model, tn:", tn)
# print("Model, fn:", fn)
model.save(model_fp)

7/7 - 1s - loss: 0.3981 - sparse_categorical_accuracy: 0.8782
Model, accuracy: 0.8781725764274597
Model, loss: 0.3980976939201355


In [68]:
# new_model = tf.keras.models.load_model(model_fp)
# new_model.summary()
model.summary()
predictions = model.predict(test_ds)
print(predictions)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 128)               8192      
_________________________________________________________________
dense_26 (Dense)             (None, 26)                3354      
Total params: 11,546
Trainable params: 11,546
Non-trainable params: 0
_________________________________________________________________
[[5.4231881e-05 1.5477355e-05 1.6339950e-06 ... 3.9241692e-07
  4.1092713e-07 2.3838633e-07]
 [4.9991505e-03 3.1701420e-05 9.7450824e-04 ... 3.8252433e-06
  3.5042567e-06 1.6668801e-06]
 [7.5976044e-04 2.4323636e-08 1.2732192e-04 ... 5.3043358e-08
  3.0541571e-08 2.1340167e-08]
 ...
 [2.0009610e-03 2.7958080e-03 1.5041308e-03 ... 6.0474281e-06
  8.0913733e-06 7.4584605e-06]
 [2.3841417e-07 8.1429757e-02 3.7986881e-04 ... 4.7569853e-07
  1.3758986e-07 4.7611678e-08]
 [1.5326575e-04 2.4476453e-06 6.9979278e-

In [69]:
df = pd.read_csv(test_fp)
count = 0
correct = 0
total = len(df)
for pred in predictions:
    actual = df.loc[count,"label"]
    prediction = np.argmax(pred)
    print("Actual:", actual, "Prediction:", prediction)
    if actual == prediction:
        correct += 1
    count += 1
print("--------------------------\n"+str(correct)+" out of "+str(total)+" predicted correctly")
print("Accuracy: "+str((correct/total)*100)+"%")

Actual: 18 Prediction: 10
Actual: 13 Prediction: 6
Actual: 9 Prediction: 11
Actual: 9 Prediction: 12
Actual: 18 Prediction: 11
Actual: 18 Prediction: 7
Actual: 6 Prediction: 11
Actual: 16 Prediction: 9
Actual: 13 Prediction: 12
Actual: 11 Prediction: 12
Actual: 3 Prediction: 0
Actual: 0 Prediction: 5
Actual: 15 Prediction: 19
Actual: 15 Prediction: 0
Actual: 19 Prediction: 5
Actual: 12 Prediction: 13
Actual: 18 Prediction: 0
Actual: 19 Prediction: 12
Actual: 9 Prediction: 0
Actual: 18 Prediction: 17
Actual: 7 Prediction: 1
Actual: 1 Prediction: 17
Actual: 5 Prediction: 12
Actual: 5 Prediction: 9
Actual: 10 Prediction: 10
Actual: 16 Prediction: 10
Actual: 8 Prediction: 7
Actual: 0 Prediction: 10
Actual: 4 Prediction: 18
Actual: 13 Prediction: 0
Actual: 17 Prediction: 7
Actual: 7 Prediction: 18
Actual: 8 Prediction: 0
Actual: 1 Prediction: 10
Actual: 8 Prediction: 10
Actual: 7 Prediction: 0
Actual: 12 Prediction: 15
Actual: 2 Prediction: 4
Actual: 10 Prediction: 12
Actual: 18 Prediction: